In [3]:
import pandas as pd
import numpy as np
import os
import glob
import shutil
from pathlib import Path
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class ExcelProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.wb = None
        self.system_folder = ""
        self.load_workbook()
    
    def load_workbook(self):
        """Загрузить Excel файл"""
        try:
            self.wb = pd.ExcelFile(self.file_path)
            print(f"Файл {self.file_path} успешно загружен")
        except Exception as e:
            print(f"Ошибка загрузки файла: {e}")
    
    def create_sheets_and_fill_headers(self):
        """Создать листы и заполнить заголовки с цветом"""
        with pd.ExcelWriter(self.file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # Список листов для создания
            sheet_names = ["Поставщикам_1", "Поставщикам_2", "Свойства", "Эталонки", "Рекласс", "Сверка", "Значения", "Фаст"]
            
            # Создаем базовые DataFrame для каждого листа
            for sheet_name in sheet_names:
                df = pd.DataFrame()
                
                if sheet_name in ["Эталонки", "Рекласс", "Сверка"]:
                    # 146 колонок с dummy
                    df = pd.DataFrame({f'col_{i}': ['dummy'] for i in range(1, 147)})
                    
                    # Специальные заголовки
                    highlights = {
                        1: "Код класса ТМЦ и услуг",
                        3: "Код КССС", 
                        4: "Краткое наименование/ru_RU",
                        7: "Полное наименование/ru_RU",
                        17: "Базовая единица измерения",
                        21: "Статус записи",
                        22: "Тип записи",
                        133: "Комментарий"
                    }
                    
                    for col, value in highlights.items():
                        if col-1 < len(df.columns):
                            df.iloc[0, col-1] = value
                
                elif sheet_name == "Значения":
                    df = pd.DataFrame({
                        'col_1': ['Код КССС'],
                        'col_2': ['свойство'], 
                        'col_3': ['значение']
                    })
                
                elif sheet_name == "Фаст":
                    df = pd.DataFrame({
                        'col_1': ['Тип записи'],
                        'col_2': ['Код КССС']
                    })
                
                elif sheet_name == "Поставщикам_1":
                    df = pd.DataFrame({'col_1': ['Код КССС']})
                
                elif sheet_name == "Поставщикам_2":
                    df = pd.DataFrame({'col_1': ['Код КССС']})
                
                elif sheet_name == "Свойства":
                    df = pd.DataFrame({
                        'col_1': ['Код свойства'],
                        'col_2': ['Наименование свойства']
                    })
                
                # Сохраняем лист
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)
            
            print("Листы созданы и заполнены")
    
    def insert_three_ranges_to_etalonki(self, range1, range2, range3):
        """Вставить три диапазона в лист Эталонки"""
        try:
            # Загружаем лист Эталонки
            df_etalonki = pd.read_excel(self.file_path, sheet_name='Эталонки', header=None)
            
            # Заменяем неразрывные пробелы
            range1 = [str(x).replace('\xa0', ' ') for x in range1]
            range2 = [str(x).replace('\xa0', ' ') for x in range2]
            range3 = [str(x).replace('\xa0', ' ') for x in range3]
            
            # Определяем максимальное количество строк
            max_rows = max(len(range1), len(range2), len(range3))
            
            # Заполняем данные начиная со второй строки
            start_row = 1  # 0-based indexing
            
            # Колонка 4 (индекс 3)
            for i, value in enumerate(range1):
                if i + start_row < len(df_etalonki):
                    df_etalonki.iloc[i + start_row, 3] = str(value)
            
            # Колонка 7 (индекс 6)
            for i, value in enumerate(range2):
                if i + start_row < len(df_etalonki):
                    df_etalonki.iloc[i + start_row, 6] = str(value)
            
            # Колонка 17 (индекс 16)
            for i, value in enumerate(range3):
                if i + start_row < len(df_etalonki):
                    df_etalonki.iloc[i + start_row, 16] = str(value)
            
            # Определяем сколько заполнено в колонке 4
            last_row = df_etalonki[3].last_valid_index()
            if last_row is None or last_row < start_row:
                print("Поле Краткое наименование в листе Эталонки пусто")
                return
            
            rows_to_fill = last_row - start_row + 1
            
            # Заполняем другие колонки
            for i in range(start_row, start_row + rows_to_fill):
                df_etalonki.iloc[i, 20] = "'00"  # колонка 21
                df_etalonki.iloc[i, 21] = "'03"  # колонка 22
                df_etalonki.iloc[i, 132] = "Эталонная запись"  # колонка 133
            
            # Сохраняем изменения
            with pd.ExcelWriter(self.file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df_etalonki.to_excel(writer, sheet_name='Эталонки', index=False, header=False)
            
            print("Данные успешно вставлены в Эталонки")
            
        except Exception as e:
            print(f"Ошибка при вставке данных: {e}")
    
    def copy_class_value_to_etalonki(self):
        """Скопировать значение класса в Эталонки"""
        try:
            # Загружаем листы
            df_props = pd.read_excel(self.file_path, sheet_name='Свойства ТМЦ', header=None)
            df_etalonki = pd.read_excel(self.file_path, sheet_name='Эталонки', header=None)
            
            # Ищем значение "Класс" во второй колонке
            class_row = df_props[df_props[1] == 'Класс'].index
            if len(class_row) == 0:
                print("Значение 'Класс' не найдено")
                return
            
            class_value = str(df_props.iloc[class_row[0] + 1, 1])
            
            # Определяем количество заполненных строк в Эталонки (колонка 4)
            start_row = 1
            last_row = df_etalonki[3].last_valid_index()
            if last_row is None or last_row < start_row:
                print("Нет данных в столбце Краткое наименование")
                return
            
            # Заполняем колонку 1
            for i in range(start_row, last_row + 1):
                df_etalonki.iloc[i, 0] = class_value
            
            # Сохраняем
            with pd.ExcelWriter(self.file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df_etalonki.to_excel(writer, sheet_name='Эталонки', index=False, header=False)
            
            print(f"Номер класса '{class_value}' скопирован")
            
        except Exception as e:
            print(f"Ошибка: {e}")
    
    def save_sheets_to_utf16_txt(self):
        """Сохранить листы в UTF-16 текстовые файлы"""
        try:
            base_path = Path(self.file_path).parent
            user_name = os.getenv('USERNAME', 'user')[:5]
            current_time = datetime.now().strftime("%H%M")
            
            sheet_mapping = {
                "Поставщикам_1": "1_Catalog_MTR_To_Catalog_SUPPLIER_POSITIONS_Import",
                "Поставщикам_2": "8_Catalog_MTR_To_Catalog_SUPPLIER_POSITIONS_Import", 
                "Эталонки": "4_Catalog_MTR_import",
                "Рекласс": "9_Catalog_supplier_positions_import",
                "Значения": "5_Catalog_MTR_property_value_import",
                "Фаст": "10_FAST"
            }
            
            for sheet_name, file_prefix in sheet_mapping.items():
                try:
                    df = pd.read_excel(self.file_path, sheet_name=sheet_name, header=None)
                    
                    # Для определенных файлов добавляем время и имя пользователя
                    if sheet_name in ["Поставщикам_2", "Рекласс"]:
                        file_name = f"{file_prefix}_{current_time}_{user_name}_{len(df)-1}.txt"
                    else:
                        file_name = f"{file_prefix}_{len(df)-1}.txt"
                    
                    file_path = base_path / file_name
                    
                    # Сохраняем как CSV с табуляцией
                    df.to_csv(file_path, sep='\t', index=False, header=False, encoding='utf-16')
                    
                    print(f"Сохранен: {file_name}")
                    
                except Exception as e:
                    print(f"Ошибка при сохранении {sheet_name}: {e}")
            
        except Exception as e:
            print(f"Ошибка: {e}")
    
    def check_and_replace_spaces(self):
        """Проверить и заменить пробелы в Эталонки"""
        try:
            df = pd.read_excel(self.file_path, sheet_name='Эталонки', header=None)
            
            columns_to_check = [3, 6]  # Колонки D и G (0-based: 3, 6)
            changes_made = False
            
            for col in columns_to_check:
                for idx, value in df[col].items():
                    if pd.notna(value) and isinstance(value, str):
                        original = value
                        modified = value
                        
                        # Замена пробелов в начале и конце
                        if modified.startswith(' '):
                            modified = '$' + modified[1:]
                            changes_made = True
                        if modified.endswith(' '):
                            modified = modified[:-1] + '$'
                            changes_made = True
                        
                        # Замена двойных пробелов
                        while '  ' in modified:
                            modified = modified.replace('  ', '$$')
                            changes_made = True
                        
                        if modified != original:
                            df.loc[idx, col] = modified
            
            if changes_made:
                with pd.ExcelWriter(self.file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    df.to_excel(writer, sheet_name='Эталонки', index=False, header=False)
                print("Пробелы заменены на '$'")
            else:
                print("Изменений не требуется")
                
        except Exception as e:
            print(f"Ошибка: {e}")
    
    def combine_utf16_files(self, folder_path=None):
        """Объединить UTF-16 файлы"""
        if folder_path is None:
            folder_path = Path(self.file_path).parent
        
        folder_path = Path(folder_path)
        output_file = folder_path / "TOTAL.txt"
        
        txt_files = list(folder_path.glob("*.txt"))
        if output_file in txt_files:
            txt_files.remove(output_file)
        
        if not txt_files:
            print("Текстовые файлы не найдены")
            return
        
        all_data = []
        first_file = True
        
        for file_path in txt_files:
            try:
                # Читаем UTF-16 файл
                with open(file_path, 'r', encoding='utf-16') as f:
                    lines = f.readlines()
                
                if first_file:
                    all_data.extend(lines)
                    first_file = False
                else:
                    # Пропускаем заголовок для последующих файлов
                    all_data.extend(lines[1:])
                    
            except Exception as e:
                print(f"Ошибка чтения {file_path}: {e}")
        
        # Сохраняем объединенный файл
        with open(output_file, 'w', encoding='utf-16') as f:
            f.writelines(all_data)
        
        print(f"Файлы объединены в: {output_file}")
    
    def export_template_sheets(self):
        """Экспорт шаблонных листов"""
        try:
            base_path = Path(self.file_path).parent
            sheet_mapping = {
                "Свойства ТМЦ": "2_Catalog_templates_TMC_Import.txt",
                "Значения свойств ТМЦ": "3_Catalog_templates_property_TMC_Values_Import.txt", 
                "Свойства ОЛ": "6_Catalog_Templates_Questionnaries_import.txt",
                "Значения свойств ОЛ": "7_Catalog_Templates_Property_Questionnaries_Values_import.txt"
            }
            
            for sheet_name, file_name in sheet_mapping.items():
                try:
                    df = pd.read_excel(self.file_path, sheet_name=sheet_name, header=None)
                    
                    # Находим строку с "Идентификатор шаблона"
                    template_row = None
                    for idx, row in df.iterrows():
                        if str(row[0]).strip().upper() == "ИДЕНТИФИКАТОР ШАБЛОНА":
                            template_row = idx
                            break
                    
                    if template_row is not None:
                        # Берем данные от этой строки до конца
                        output_df = df.iloc[template_row:]
                        
                        # Заменяем неразрывные пробелы
                        for col in output_df.columns:
                            output_df[col] = output_df[col].apply(
                                lambda x: str(x).replace('\xa0', ' ') if pd.notna(x) else x
                            )
                        
                        # Сохраняем
                        file_path = base_path / file_name
                        output_df.to_csv(file_path, sep='\t', index=False, header=False, encoding='utf-16')
                        print(f"Сохранен: {file_name}")
                    
                except Exception as e:
                    print(f"Ошибка при экспорте {sheet_name}: {e}")
                    
        except Exception as e:
            print(f"Ошибка: {e}")


In [ ]:
def main():
    # Инициализация процессора
    processor = ExcelProcessor("шаблон\Наушники противошумные_19.xlsx")
    
    # Создание листов
    processor.create_sheets_and_fill_headers()
    
    # Пример вставки данных (нужно подготовить данные)
    # range1 = ["краткое1", "краткое2", "краткое3"]
    # range2 = ["полное1", "полное2", "полное3"] 
    # range3 = ["ЕИ1", "ЕИ2", "ЕИ3"]
    # processor.insert_three_ranges_to_etalonki(range1, range2, range3)
    
    # Копирование класса
    processor.copy_class_value_to_etalonki()
    
    # Проверка пробелов
    processor.check_and_replace_spaces()
    
    # Сохранение текстовых файлов
    processor.save_sheets_to_utf16_txt()
    
    # Экспорт шаблонов
    processor.export_template_sheets()
    
    # Объединение файлов
    processor.combine_utf16_files()

if __name__ == "__main__":
    main()


In [6]:
def main():
    # Инициализация процессора
    processor = ExcelProcessor("шаблон\Наушники противошумные_19.xlsx")
    
    # 1. Создание листов (обязательный первый шаг)
    processor.create_sheets_and_fill_headers()
    
    # 2. Подготовка данных для вставки в Эталонки
    # range1 - КРАТКИЕ НАИМЕНОВАНИЯ (колонка 4 в Эталонках)
    # range2 - ПОЛНЫЕ НАИМЕНОВАНИЯ (колонка 7 в Эталонках) 
    # range3 - БАЗОВЫЕ ЕДИНИЦЫ ИЗМЕРЕНИЯ (колонка 17 в Эталонках)
    
    # ПРИМЕР 1: Данные из списков
    range1 = ["Болт М6х20", "Гайка М8", "Шайба 8.4", "Винт саморез"]  # Краткие наименования
    range2 = ["Болт шестигранный М6х20", "Гайка шестигранная М8", "Шайба пружинная 8.4", "Винт самонарезающий"]  # Полные наименования
    range3 = ["шт", "шт", "шт", "шт"]  # Единицы измерения
    
    # ПРИМЕР 2: Загрузка из CSV/Excel файла
    def load_data_from_file(file_path, column_name, start_row=0):
        """Загрузить данные из столбца файла"""
        df = pd.read_excel(file_path)  # или pd.read_csv(file_path)
        return df[column_name].dropna().tolist()[start_row:]
    
    # Загрузка из внешнего файла
    # range1 = load_data_from_file("source_data.xlsx", "Краткое наименование", start_row=5)
    # range2 = load_data_from_file("source_data.xlsx", "Полное наименование", start_row=5) 
    # range3 = load_data_from_file("source_data.xlsx", "Единица измерения", start_row=5)
    
    # ПРИМЕР 3: Данные из диапазона существующего Excel
    def get_range_from_sheet(file_path, sheet_name, col_letter, start_row, end_row=None):
        """Получить диапазон данных из листа"""
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        start_idx = start_row - 1  # pandas 0-based indexing
        if end_row:
            end_idx = end_row
        else:
            # Автоматически определяем последнюю заполненную строку
            end_idx = df[col_letter].last_valid_index() + 1
        
        return df.iloc[start_idx:end_idx, col_letter].dropna().tolist()
    
    # Пример получения данных с листа "Источник" 
    # range1 = get_range_from_sheet("your_file.xlsx", "Источник", 2, 5)  # колонка B, с 5й строки
    # range2 = get_range_from_sheet("your_file.xlsx", "Источник", 3, 5)  # колонка C, с 5й строки  
    # range3 = get_range_from_sheet("your_file.xlsx", "Источник", 4, 5)  # колонка D, с 5й строки
    
    # 3. Вставка данных в Эталонки
    processor.insert_three_ranges_to_etalonki(range1, range2, range3)
    
    # 4. Остальные операции
    processor.copy_class_value_to_etalonki()
    processor.check_and_replace_spaces()
    processor.save_sheets_to_utf16_txt()
    processor.export_template_sheets()
    processor.combine_utf16_files()

# Конкретный пример работы с реальными данными
def real_world_example():
    """Пример работы с реальными данными из файла поставщика"""
    
    processor = ExcelProcessor("обработка_ТМЦ.xlsx")
    processor.create_sheets_and_fill_headers()
    
    # ДАННЫЕ ИЗ ФАЙЛА ПОСТАВЩИКА (пример структуры)
    # Предположим, что у нас есть файл supplier_data.xlsx со следующими колонками:
    # A: Артикул, B: Краткое описание, C: Полное описание, D: Ед.изм.
    
    # Загружаем данные из файла поставщика
    supplier_df = pd.read_excel("supplier_data.xlsx")
    
    # Извлекаем нужные колонки (начиная с 6й строки, если заголовок в 5й строке)
    range1 = supplier_df["Краткое наименование нормализованное"].iloc[5:].dropna().tolist()  # с 6й строки
    range2 = supplier_df["Полное наименование нормализованное"].iloc[5:].dropna().tolist()    # с 6й строки  
    range3 = supplier_df["Базовая единица измерения МТР (цел.)"].iloc[5:].dropna().tolist()           # с 6й строки
    
    print(f"Загружено {len(range1)} позиций для обработки")
    
    # Вставляем в Эталонки
    processor.insert_three_ranges_to_etalonki(range1, range2, range3)
    
    # Выполняем остальные операции
    processor.copy_class_value_to_etalonki()
    processor.check_and_replace_spaces()
    processor.save_sheets_to_utf16_txt()
    
    print("Обработка завершена!")

# Автоматическое определение диапазонов
def auto_detect_ranges():
    """Автоматическое определение диапазонов данных"""
    
    processor = ExcelProcessor("обработка_ТМЦ.xlsx")
    
    # Предположим, данные находятся на листе "Данные поставщика"
    data_df = pd.read_excel("обработка_ТМЦ.xlsx", sheet_name="Данные поставщика", header=None)
    
    # Находим заголовок "Краткое наименование" и берем все данные под ним
    def find_column_by_header(df, header_text):
        """Найти колонку по заголовку"""
        for col in df.columns:
            for idx, value in df[col].items():
                if str(value).strip() == header_text:
                    return col, idx
        return None, None
    
    # Находим колонки
    col_short, header_row_short = find_column_by_header(data_df, "Краткое наименование нормализованное")
    col_full, header_row_full = find_column_by_header(data_df, "Полное наименование нормализованное") 
    col_unit, header_row_unit = find_column_by_header(data_df, "Базовая единица измерения МТР (цел.)")
    
    if all([col_short, col_full, col_unit]):
        # Берем данные с строки ниже заголовка до последней заполненной
        range1 = data_df[col_short].iloc[header_row_short+2:].dropna().tolist()
        range2 = data_df[col_full].iloc[header_row_full+2:].dropna().tolist()
        range3 = data_df[col_unit].iloc[header_row_unit+2:].dropna().tolist()
        
        print(f"Автоматически обнаружено: {len(range1)} позиций")
        
        processor.insert_three_ranges_to_etalonki(range1, range2, range3)
        # ... остальные операции

# Проверка данных перед вставкой
def validate_data_before_insert():
    """Проверка данных перед вставкой"""
    
    range1 = ["Болт М6х20", "Гайка М8", "Шайба 8.4"]
    range2 = ["Болт шестигранный М6х20", "Гайка шестигранная М8", "Шайба пружинная 8.4"] 
    range3 = ["шт", "шт", "шт"]
    
    # Проверка длин массивов
    if len(range1) != len(range2) or len(range1) != len(range3):
        print("ОШИБКА: Диапазоны разной длины!")
        return
    
    # Проверка на пустые значения
    if not all(range1) or not all(range2) or not all(range3):
        print("ОШИБКА: Есть пустые значения!")
        return
    
    print("Данные прошли проверку, можно вставлять")
    
    processor = ExcelProcessor("обработка_ТМЦ.xlsx")
    processor.insert_three_ranges_to_etalonki(range1, range2, range3)




In [ ]:
if __name__ == "__main__":
    # Выберите нужный пример:
    #main()               # Базовый пример
    # real_world_example() # Пример с реальными данными  
    auto_detect_ranges() # Автоматическое определение
    #validate_data_before_insert() # С проверкой данных

## Сравнение колонок

In [1]:
# additional_functions.py
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

class DataValidator:
    def __init__(self, file_path):
        self.file_path = file_path
    
    def check_properties_tmc(self):
        """Проверить свойства ТМЦ"""
        try:
            df_tmc = pd.read_excel(self.file_path, sheet_name='Свойства ТМЦ', header=None)
            df_props = pd.read_excel(self.file_path, sheet_name='Свойства', header=None)
            
            # Находим строку с "Свойство" в колонке 9
            prop_row = None
            for idx, row in df_tmc.iterrows():
                if str(row[8]).strip() == "Свойство":  # колонка 9 (индекс 8)
                    prop_row = idx
                    break
            
            if prop_row is None:
                print("Свойство не найдено")
                return
            
            # Собираем существующие свойства
            existing_props = set(str(x).strip() for x in df_props[0] if pd.notna(x))
            
            # Проверяем свойства ниже найденной строки
            not_found = []
            for idx in range(prop_row + 1, len(df_tmc)):
                prop_value = str(df_tmc.iloc[idx, 8]).strip()
                if prop_value and prop_value not in existing_props:
                    not_found.append(prop_value)
            
            if not_found:
                print(f"Ненайденные свойства: {not_found}")
            else:
                print("Все свойства найдены")
                
        except Exception as e:
            print(f"Ошибка: {e}")
    
    def highlight_duplicates(self, sheet_name, col1, col2):
        """Подсветить дубликаты по двум колонкам"""
        try:
            df = pd.read_excel(self.file_path, sheet_name=sheet_name, header=None)
            
            # Находим строку с "Свойство"
            prop_row = None
            for idx, row in df.iterrows():
                if str(row[col1-1]).strip() == "Свойство":
                    prop_row = idx
                    break
            
            if prop_row is None:
                print("Свойство не найдено")
                return
            
            # Проверяем дубликаты
            pairs_count = {}
            start_row = prop_row + 1
            
            for idx in range(start_row, len(df)):
                val1 = str(df.iloc[idx, col1-1]).strip()
                val2 = str(df.iloc[idx, col2-1]).strip()
                
                if val1 and val2:
                    key = f"{val1}||{val2}"
                    pairs_count[key] = pairs_count.get(key, 0) + 1
            
            # Помечаем дубликаты
            duplicates_found = False
            for idx in range(start_row, len(df)):
                val1 = str(df.iloc[idx, col1-1]).strip()
                val2 = str(df.iloc[idx, col2-1]).strip()
                
                if val1 and val2:
                    key = f"{val1}||{val2}"
                    if pairs_count[key] > 1:
                        duplicates_found = True
                        # В реальной реализации здесь нужно сохранить форматирование
            
            if duplicates_found:
                print("Найдены дубликаты")
            else:
                print("Дубликатов не найдено")
                
        except Exception as e:
            print(f"Ошибка: {e}")
    
    def similarity_score(self, s1, s2):
        """Вычислить схожесть строк"""
        return SequenceMatcher(None, s1, s2).ratio()


In [24]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import re
import warnings
warnings.filterwarnings('ignore')

class ExcelComparator:
    def __init__(self, file_path):
        self.file_path = file_path
        self.wb = None
        
    def _clean_column_names(self, df):
        """Очистить названия колонок от недопустимых символов"""
        clean_columns = []
        for col in df.columns:
            # Заменяем недопустимые символы на подчеркивания
            clean_col = re.sub(r'[\[\]\{\}\(\)\-\+\=\/\*\.\,\;\:\!\?\&\|\<\>\'\"]', '_', str(col))
            # Убираем множественные подчеркивания
            clean_col = re.sub(r'_+', '_', clean_col)
            # Убираем подчеркивания в начале и конце
            clean_col = clean_col.strip('_')
            clean_columns.append(clean_col)
        df.columns = clean_columns
        return df
    
    def compare_sheets(self, sheet1_name, sheet2_name, key_column, output_sheet_name="Сравнение"):
        """
        Сравнить два листа по ключевой колонке
        
        Args:
            sheet1_name (str): Название первого листа
            sheet2_name (str): Название второго листа  
            key_column (str): Название ключевой колонки для связи
            output_sheet_name (str): Название листа для результатов
        """
        
        try:
            # Загружаем данные
            df1 = pd.read_excel(self.file_path, sheet_name=sheet1_name)
            df2 = pd.read_excel(self.file_path, sheet_name=sheet2_name)
            
            print(f"Лист 1: {len(df1)} строк, {len(df1.columns)} колонок")
            print(f"Лист 2: {len(df2)} строк, {len(df2.columns)} колонок")
            
            # Очищаем названия колонок
            df1 = self._clean_column_names(df1)
            df2 = self._clean_column_names(df2)
            key_column_clean = self._clean_column_names(pd.DataFrame(columns=[key_column])).columns[0]
            
            print(f"Очищенные названия колонок в листе 1: {list(df1.columns)}")
            print(f"Очищенные названия колонок в листе 2: {list(df2.columns)}")
            
            # Находим общие колонки (кроме ключевой)
            common_columns = list(set(df1.columns) & set(df2.columns) - {key_column_clean})
            print(f"Общие колонки для сравнения: {common_columns}")
            
            if not common_columns:
                print("Нет общих колонок для сравнения!")
                return None
            
            if key_column_clean not in df1.columns or key_column_clean not in df2.columns:
                print(f"Ключевая колонка '{key_column_clean}' не найдена в обоих листах!")
                return None
            
            # Объединяем данные по ключевой колонке
            merged_df = pd.merge(df1, df2, on=key_column_clean, how='outer', 
                                suffixes=('_sheet1', '_sheet2'), indicator=True)
            
            print(f"После объединения: {len(merged_df)} строк")
            
            # Создаем DataFrame для результатов
            result_data = []
            
            # Подготавливаем структуру для результатов
            for idx, row in merged_df.iterrows():
                result_row = {}
                key_value = row[key_column_clean]
                result_row[key_column_clean] = key_value
                
                # Добавляем информацию о наличии в обоих листах
                merge_status = row['_merge']
                result_row['Статус'] = merge_status
                
                # Считаем количество расхождений
                diff_count = 0
                
                # Сравниваем каждую общую колонку
                for col in common_columns:
                    col1 = f"{col}_sheet1"
                    col2 = f"{col}_sheet2"
                    
                    val1 = row[col1] if col1 in row else None
                    val2 = row[col2] if col2 in row else None
                    
                    # Проверяем на расхождения
                    if pd.isna(val1) and pd.isna(val2):
                        # Оба значения NaN - считаем равными
                        are_different = False
                    elif pd.isna(val1) or pd.isna(val2):
                        # Один NaN, другой не NaN - разные
                        are_different = True
                    else:
                        # Оба не NaN - сравниваем значения
                        are_different = str(val1) != str(val2)
                    
                    # Сохраняем значения и флаг различия
                    result_row[f'{col}_sheet1'] = val1
                    result_row[f'{col}_sheet2'] = val2
                    result_row[f'{col}_diff'] = are_different
                    
                    if are_different:
                        diff_count += 1
                
                result_row['Количество_расхождений'] = diff_count
                result_data.append(result_row)
            
            # Создаем итоговый DataFrame
            result_df = pd.DataFrame(result_data)
            
            # Сортируем по количеству расхождений (по убыванию)
            result_df = result_df.sort_values('Количество_расхождений', ascending=False)
            
            # Сохраняем результаты в новый файл, чтобы не повредить исходный
            output_file = self.file_path.replace('.xlsx', '_comparison.xlsx')
            
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                result_df.to_excel(writer, sheet_name=output_sheet_name, index=False)
            
            # Применяем цветовое форматирование
            self._apply_color_formatting(output_file, output_sheet_name, common_columns, key_column_clean)
            
            print(f"Сравнение завершено. Результаты сохранены в файл '{output_file}'")
            print(f"Всего строк: {len(result_df)}")
            print(f"Строк с расхождениями: {len(result_df[result_df['Количество_расхождений'] > 0])}")
            
            return result_df, output_file
            
        except Exception as e:
            print(f"Ошибка при сравнении: {e}")
            import traceback
            traceback.print_exc()
            return None, None
    def create_column_report(self, sheet1_name, sheet2_name, output_sheet_name="Отчет_по_колонкам"):
        """Создать отчет по несовпадающим колонкам"""
        return compare_column_names(self.file_path, sheet1_name, sheet2_name, output_sheet_name)
    
    def full_comparison(self, sheet1_name, sheet2_name, key_column):
        """
        Полное сравнение: отчет по колонкам + сравнение данных
        """
        print("=== НАЧАЛО ПОЛНОГО СРАВНЕНИЯ ===")
        
        # 1. Сначала создаем отчет по колонкам
        print("\n1. АНАЛИЗ КОЛОНОК:")
        column_report, column_file = self.create_column_report(sheet1_name, sheet2_name)
        
        # 2. Затем сравниваем данные
        print("\n2. СРАВНЕНИЕ ДАННЫХ:")
        data_report, data_file = self.compare_sheets(sheet1_name, sheet2_name, key_column)
        
        print("\n=== ПОЛНОЕ СРАВНЕНИЕ ЗАВЕРШЕНО ===")
        print(f"Отчет по колонкам: {column_file}")
        print(f"Отчет по данным: {data_file}")
        
        return {
            'column_report': column_report,
            'column_file': column_file,
            'data_report': data_report,
            'data_file': data_file
        }



    def _apply_color_formatting(self, file_path, sheet_name, common_columns, key_column):
        """Применить цветовое форматирование к расхождениям"""
        
        try:
            # Загружаем книгу для форматирования
            book = load_workbook(file_path)
            sheet = book[sheet_name]
            
            # Розовая заливка для различий
            pink_fill = PatternFill(start_color='FFB6C1', end_color='FFB6C1', fill_type='solid')
            
            # Находим индексы колонок
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            # Создаем mapping названий колонок к индексам
            col_mapping = {}
            for idx, col_name in enumerate(df.columns, 1):  # Excel columns start from 1
                col_mapping[col_name] = idx
            
            # Применяем форматирование для каждой строки
            for row_idx in range(2, len(df) + 2):  # +2 because Excel rows start from 1 and we have header
                for col in common_columns:
                    diff_col_name = f'{col}_diff'
                    if diff_col_name in col_mapping:
                        try:
                            diff_cell_value = sheet.cell(row=row_idx, column=col_mapping[diff_col_name]).value
                            
                            if diff_cell_value == True:  # Если есть расхождение
                                # Закрашиваем соответствующие ячейки с значениями
                                col1_idx = col_mapping.get(f'{col}_sheet1')
                                col2_idx = col_mapping.get(f'{col}_sheet2')
                                
                                if col1_idx:
                                    sheet.cell(row=row_idx, column=col1_idx).fill = pink_fill
                                if col2_idx:
                                    sheet.cell(row=row_idx, column=col2_idx).fill = pink_fill
                        except Exception as e:
                            print(f"Ошибка при форматировании строки {row_idx}, колонка {col}: {e}")
                            continue
            
            # Скрываем колонки с флагами различий
            for col in common_columns:
                diff_col_name = f'{col}_diff'
                if diff_col_name in col_mapping:
                    col_letter = self._get_column_letter(col_mapping[diff_col_name])
                    sheet.column_dimensions[col_letter].hidden = True
            
            # Автоматически подгоняем ширину колонок
            for column in sheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if cell.value is not None and len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = min(max_length + 2, 50)  # Ограничиваем максимальную ширину
                sheet.column_dimensions[column_letter].width = adjusted_width
            
            book.save(file_path)
            print("Цветовое форматирование применено")
            
        except Exception as e:
            print(f"Ошибка при применении форматирования: {e}")
    
    def _get_column_letter(self, col_idx):
        """Конвертировать индекс колонки в букву"""
        letters = ''
        while col_idx > 0:
            col_idx, remainder = divmod(col_idx - 1, 26)
            letters = chr(65 + remainder) + letters
        return letters

# Упрощенная версия для быстрого использования
def safe_compare_sheets(file_path, sheet1, sheet2, key_column):
    """
    Безопасное сравнение двух листов с обработкой ошибок
    
    Args:
        file_path (str): путь к Excel файлу
        sheet1 (str): название первого листа
        sheet2 (str): название второго листа
        key_column (str): ключевая колонка для сравнения
    """
    
    print(f"Начинаем сравнение: {sheet1} vs {sheet2} по колонке '{key_column}'")
    
    comparator = ExcelComparator(file_path)
    result_df, output_file = comparator.compare_sheets(
        sheet1_name=sheet1,
        sheet2_name=sheet2,
        key_column=key_column,
        output_sheet_name="Результат_сравнения"
    )
    
    if result_df is not None:
        print("\n=== РЕЗУЛЬТАТЫ СРАВНЕНИЯ ===")
        print(f"Файл с результатами: {output_file}")
        print(f"Всего строк: {len(result_df)}")
        
        # Статистика
        total_diff = len(result_df[result_df['Количество_расхождений'] > 0])
        only_in_sheet1 = len(result_df[result_df['Статус'] == 'left_only'])
        only_in_sheet2 = len(result_df[result_df['Статус'] == 'right_only'])
        in_both = len(result_df[result_df['Статус'] == 'both'])
        
        print(f"Строк с расхождениями: {total_diff}")
        print(f"Только в {sheet1}: {only_in_sheet1}")
        print(f"Только в {sheet2}: {only_in_sheet2}")
        print(f"В обоих листах: {in_both}")
        
        # Показываем топ-5 строк с наибольшими расхождениями
        if total_diff > 0:
            print("\nТоп-5 строк с наибольшими расхождениями:")
            top_diffs = result_df.head().copy()
            for idx, row in top_diffs.iterrows():
                print(f"  {row[key_column]}: {row['Количество_расхождений']} расхождений")
    
    return result_df, output_file






In [ ]:
# Пример использования
if __name__ == "__main__":
    # Укажите путь к вашему файлу
    file_path = "шаблон\тест_сверка2.xlsx"
    
    # Укажите названия листов и ключевую колонку
    sheet1_name = "КССС1"  # Замените на реальное название
    sheet2_name = "КССС2"    # Замените на реальное название  
    key_column = "CSCD_ID"   # Замените на реальное название ключевой колонки
    
    try:
        result_df, output_file = safe_compare_sheets(
            file_path=file_path,
            sheet1=sheet1_name,
            sheet2=sheet2_name,
            key_column=key_column
        )
        
    except Exception as e:
        print(f"Критическая ошибка: {e}")

## Сравнение 2

In [25]:
def compare_column_names(file_path, sheet1_name, sheet2_name, output_sheet_name="Отчет_по_колонкам"):
    """
    Сравнить названия колонок между двумя листами и создать отчет о несовпадающих колонках
    
    Args:
        file_path (str): путь к Excel файлу
        sheet1_name (str): название первого листа
        sheet2_name (str): название второго листа
        output_sheet_name (str): название листа для отчета
    
    Returns:
        pd.DataFrame: DataFrame с отчетом о колонках
        str: путь к файлу с отчетом
    """
    
    try:
        # Загружаем данные
        df1 = pd.read_excel(file_path, sheet_name=sheet1_name)
        df2 = pd.read_excel(file_path, sheet_name=sheet2_name)
        
        print(f"Сравнение колонок: {sheet1_name} vs {sheet2_name}")
        print(f"Колонок в {sheet1_name}: {len(df1.columns)}")
        print(f"Колонок в {sheet2_name}: {len(df2.columns)}")
        
        # Получаем множества колонок
        cols1 = set(df1.columns)
        cols2 = set(df2.columns)
        
        # Находим различия
        only_in_sheet1 = cols1 - cols2
        only_in_sheet2 = cols2 - cols1
        common_columns = cols1 & cols2
        
        print(f"Общие колонки: {len(common_columns)}")
        print(f"Только в {sheet1_name}: {len(only_in_sheet1)}")
        print(f"Только в {sheet2_name}: {len(only_in_sheet2)}")
        
        # Создаем отчет
        report_data = []
        
        # Добавляем общие колонки
        for col in sorted(common_columns):
            report_data.append({
                'Название_колонки': col,
                'Статус': 'ОБЩАЯ',
                'Лист1_тип': str(df1[col].dtype),
                'Лист2_тип': str(df2[col].dtype),
                'Лист1_непустых': df1[col].count(),
                'Лист2_непустых': df2[col].count(),
                'Примечание': '✓'
            })
        
        # Добавляем колонки только в первом листе
        for col in sorted(only_in_sheet1):
            report_data.append({
                'Название_колонки': col,
                'Статус': f'ТОЛЬКО В {sheet1_name}',
                'Лист1_тип': str(df1[col].dtype),
                'Лист2_тип': 'НЕТ',
                'Лист1_непустых': df1[col].count(),
                'Лист2_непустых': 0,
                'Примечание': f'Отсутствует в {sheet2_name}'
            })
        
        # Добавляем колонки только во втором листе
        for col in sorted(only_in_sheet2):
            report_data.append({
                'Название_колонки': col,
                'Статус': f'ТОЛЬКО В {sheet2_name}',
                'Лист1_тип': 'НЕТ',
                'Лист2_тип': str(df2[col].dtype),
                'Лист1_непустых': 0,
                'Лист2_непустых': df2[col].count(),
                'Примечание': f'Отсутствует в {sheet1_name}'
            })
        
        # Создаем DataFrame отчета
        report_df = pd.DataFrame(report_data)
        
        # Сохраняем отчет в новый файл
        output_file = file_path.replace('.xlsx', '_column_report.xlsx')
        
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            report_df.to_excel(writer, sheet_name=output_sheet_name, index=False)
        
        # Применяем форматирование к отчету
        _format_column_report(output_file, output_sheet_name, sheet1_name, sheet2_name)
        
        print(f"Отчет по колонкам сохранен в: {output_file}")
        
        # Выводим сводную информацию
        _print_column_summary(report_df, sheet1_name, sheet2_name)
        
        return report_df, output_file
        
    except Exception as e:
        print(f"Ошибка при создании отчета по колонкам: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def _format_column_report(file_path, sheet_name, sheet1_name, sheet2_name):
    """Применить форматирование к отчету по колонкам"""
    
    try:
        book = load_workbook(file_path)
        sheet = book[sheet_name]
        
        # Цвета для разных статусов
        common_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')  # Зеленый
        only_sheet1_fill = PatternFill(start_color='FFEB9C', end_color='FFEB9C', fill_type='solid')  # Желтый
        only_sheet2_fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')  # Красный
        
        # Находим индексы колонок
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Применяем цветовое форматирование
        for row_idx in range(2, len(df) + 2):  # Пропускаем заголовок
            status_cell = sheet.cell(row=row_idx, column=2)  # Колонка "Статус"
            status_value = status_cell.value
            
            if status_value == 'ОБЩАЯ':
                fill_color = common_fill
            elif f'ТОЛЬКО В {sheet1_name}' in str(status_value):
                fill_color = only_sheet1_fill
            elif f'ТОЛЬКО В {sheet2_name}' in str(status_value):
                fill_color = only_sheet2_fill
            else:
                fill_color = None
            
            if fill_color:
                # Закрашиваем всю строку
                for col_idx in range(1, len(df.columns) + 1):
                    sheet.cell(row=row_idx, column=col_idx).fill = fill_color
        
        # Автоподгонка ширины колонок
        for column in sheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if cell.value is not None and len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 50)
            sheet.column_dimensions[column_letter].width = adjusted_width
        
        book.save(file_path)
        print("Форматирование отчета применено")
        
    except Exception as e:
        print(f"Ошибка при форматировании отчета: {e}")

def _print_column_summary(report_df, sheet1_name, sheet2_name):
    """Вывести сводную информацию по колонкам"""
    
    if report_df is None:
        return
    
    total_columns = len(report_df)
    common_columns = len(report_df[report_df['Статус'] == 'ОБЩАЯ'])
    only_sheet1 = len(report_df[report_df['Статус'] == f'ТОЛЬКО В {sheet1_name}'])
    only_sheet2 = len(report_df[report_df['Статус'] == f'ТОЛЬКО В {sheet2_name}'])
    
    print("\n" + "="*60)
    print("СВОДКА ПО КОЛОНКАМ")
    print("="*60)
    print(f"Всего уникальных колонок: {total_columns}")
    print(f"Общие колонки: {common_columns}")
    print(f"Только в {sheet1_name}: {only_sheet1}")
    print(f"Только в {sheet2_name}: {only_sheet2}")
    
    if only_sheet1 > 0:
        print(f"\nКолонки, которые есть ТОЛЬКО в '{sheet1_name}':")
        for col in report_df[report_df['Статус'] == f'ТОЛЬКО В {sheet1_name}']['Название_колонки'].head(10):
            print(f"  - {col}")
        if only_sheet1 > 10:
            print(f"  ... и еще {only_sheet1 - 10} колонок")
    
    if only_sheet2 > 0:
        print(f"\nКолонки, которые есть ТОЛЬКО в '{sheet2_name}':")
        for col in report_df[report_df['Статус'] == f'ТОЛЬКО В {sheet2_name}']['Название_колонки'].head(10):
            print(f"  - {col}")
        if only_sheet2 > 10:
            print(f"  ... и еще {only_sheet2 - 10} колонок")

# Обновленная версия основного класса с интеграцией отчета по колонкам


# Дополнения к существующему классу ExcelComparator (добавить эти методы)
def enhanced_compare_sheets(self, sheet1_name, sheet2_name, key_column, output_sheet_name="Сравнение"):
    """
    Улучшенное сравнение с предварительным анализом колонок
    """
    # Сначала показываем отчет по колонкам
    print("=== ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ КОЛОНОК ===")
    column_report, _ = self.create_column_report(sheet1_name, sheet2_name)
    
    # Затем выполняем обычное сравнение
    print("\n=== СРАВНЕНИЕ ДАННЫХ ===")
    return self.compare_sheets(sheet1_name, sheet2_name, key_column, output_sheet_name)

# Добавляем метод к классу
ExcelComparator.enhanced_compare_sheets = enhanced_compare_sheets

# Функция для быстрого использования отчета по колонкам
def quick_column_analysis(file_path, sheet1, sheet2):
    """
    Быстрый анализ колонок между двумя листами
    """
    print(f"Быстрый анализ колонок: {sheet1} vs {sheet2}")
    
    comparator = ExcelComparator(file_path)
    report_df, output_file = comparator.create_column_report(sheet1, sheet2)
    
    return report_df, output_file


=== ТОЛЬКО ОТЧЕТ ПО КОЛОНКАМ ===
Быстрый анализ колонок: КССС1 vs КССС2
Сравнение колонок: КССС1 vs КССС2
Колонок в КССС1: 13
Колонок в КССС2: 13
Общие колонки: 12
Только в КССС1: 1
Только в КССС2: 1
Форматирование отчета применено
Отчет по колонкам сохранен в: шаблон\тест_сверка2_column_report.xlsx

СВОДКА ПО КОЛОНКАМ
Всего уникальных колонок: 14
Общие колонки: 12
Только в КССС1: 1
Только в КССС2: 1

Колонки, которые есть ТОЛЬКО в 'КССС1':
  - Колонка тест1

Колонки, которые есть ТОЛЬКО в 'КССС2':
  - Колонка тест2

=== ПОЛНОЕ СРАВНЕНИЕ ===
=== НАЧАЛО ПОЛНОГО СРАВНЕНИЯ ===

1. АНАЛИЗ КОЛОНОК:
Сравнение колонок: КССС1 vs КССС2
Колонок в КССС1: 13
Колонок в КССС2: 13
Общие колонки: 12
Только в КССС1: 1
Только в КССС2: 1
Форматирование отчета применено
Отчет по колонкам сохранен в: шаблон\тест_сверка2_column_report.xlsx

СВОДКА ПО КОЛОНКАМ
Всего уникальных колонок: 14
Общие колонки: 12
Только в КССС1: 1
Только в КССС2: 1

Колонки, которые есть ТОЛЬКО в 'КССС1':
  - Колонка тест1

Колонки

In [ ]:
# Пример использования
if __name__ == "__main__":
    file_path = "шаблон\тест_сверка2.xlsx"
    
    # 1. Только отчет по колонкам
    print("=== ТОЛЬКО ОТЧЕТ ПО КОЛОНКАМ ===")
    column_report, column_file = quick_column_analysis(
        file_path=file_path,
        sheet1="КССС1",
        sheet2="КССС2"
    )
    
    # 2. Полное сравнение (колонки + данные)
    print("\n=== ПОЛНОЕ СРАВНЕНИЕ ===")
    comparator = ExcelComparator(file_path)
    full_result = comparator.full_comparison(
        sheet1_name="КССС1",
        sheet2_name="КССС2", 
        key_column="CSCD_ID"
    )

## Сравнение 3

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import re
import warnings
import io
import sys
from datetime import datetime

warnings.filterwarnings('ignore')

class ExcelComparator:
    def __init__(self, file_path):
        self.file_path = file_path
        self.log_messages = []
        self._setup_logging()
    
    def _setup_logging(self):
        """Настройка перехвата print сообщений"""
        self.original_stdout = sys.stdout
        self.log_capture = io.StringIO()
        sys.stdout = self.log_capture
    
    def _log_message(self, message):
        """Добавить сообщение в лог"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        self.log_messages.append(log_entry)
        self.original_stdout.write(log_entry + '\n')
    
    def _restore_stdout(self):
        """Восстановить стандартный вывод"""
        sys.stdout = self.original_stdout
    
    def _clean_column_names(self, df):
        """Очистить названия колонок от недопустимых символов"""
        clean_columns = []
        for col in df.columns:
            # Заменяем недопустимые символы на подчеркивания
            clean_col = re.sub(r'[\[\]\{\}\(\)\-\+\=\/\*\.\,\;\:\!\?\&\|\<\>\'\"]', '_', str(col))
            # Убираем множественные подчеркивания
            clean_col = re.sub(r'_+', '_', clean_col)
            # Убираем подчеркивания в начале и конце
            clean_col = clean_col.strip('_')
            clean_columns.append(clean_col)
        df.columns = clean_columns
        return df

    def compare_column_names(self, sheet1_name, sheet2_name):
        """
        Сравнить названия колонок между двумя листами и создать отчет о несовпадающих колонках
        """
        
        try:
            self._log_message(f"Начинаем сравнение колонок: {sheet1_name} vs {sheet2_name}")
            
            # Загружаем данные
            df1 = pd.read_excel(self.file_path, sheet_name=sheet1_name)
            df2 = pd.read_excel(self.file_path, sheet_name=sheet2_name)
            
            self._log_message(f"Колонок в {sheet1_name}: {len(df1.columns)}")
            self._log_message(f"Колонок в {sheet2_name}: {len(df2.columns)}")
            
            # Получаем множества колонок
            cols1 = set(df1.columns)
            cols2 = set(df2.columns)
            
            # Находим различия
            only_in_sheet1 = cols1 - cols2
            only_in_sheet2 = cols2 - cols1
            common_columns = cols1 & cols2
            
            self._log_message(f"Общие колонки: {len(common_columns)}")
            self._log_message(f"Только в {sheet1_name}: {len(only_in_sheet1)}")
            self._log_message(f"Только в {sheet2_name}: {len(only_in_sheet2)}")
            
            # Создаем отчет
            report_data = []
            
            # Добавляем общие колонки
            for col in sorted(common_columns):
                report_data.append({
                    'Название_колонки': col,
                    'Статус': 'ОБЩАЯ',
                    'Лист1_тип': str(df1[col].dtype),
                    'Лист2_тип': str(df2[col].dtype),
                    'Лист1_непустых': df1[col].count(),
                    'Лист2_непустых': df2[col].count(),
                    'Примечание': '✓'
                })
            
            # Добавляем колонки только в первом листе
            for col in sorted(only_in_sheet1):
                report_data.append({
                    'Название_колонки': col,
                    'Статус': f'ТОЛЬКО В {sheet1_name}',
                    'Лист1_тип': str(df1[col].dtype),
                    'Лист2_тип': 'НЕТ',
                    'Лист1_непустых': df1[col].count(),
                    'Лист2_непустых': 0,
                    'Примечание': f'Отсутствует в {sheet2_name}'
                })
            
            # Добавляем колонки только во втором листе
            for col in sorted(only_in_sheet2):
                report_data.append({
                    'Название_колонки': col,
                    'Статус': f'ТОЛЬКО В {sheet2_name}',
                    'Лист1_тип': 'НЕТ',
                    'Лист2_тип': str(df2[col].dtype),
                    'Лист1_непустых': 0,
                    'Лист2_непустых': df2[col].count(),
                    'Примечание': f'Отсутствует в {sheet1_name}'
                })
            
            # Создаем DataFrame отчета
            report_df = pd.DataFrame(report_data)
            
            self._log_message("Отчет по колонкам успешно создан")
            
            return report_df
            
        except Exception as e:
            self._log_message(f"Ошибка при создании отчета по колонкам: {e}")
            import traceback
            traceback.print_exc()
            return None

    def compare_sheets(self, sheet1_name, sheet2_name, key_column):
        """
        Сравнить два листа по ключевой колонке
        """
        
        try:
            self._log_message(f"Начинаем сравнение данных: {sheet1_name} vs {sheet2_name} по колонке '{key_column}'")
            
            # Загружаем данные
            df1 = pd.read_excel(self.file_path, sheet_name=sheet1_name)
            df2 = pd.read_excel(self.file_path, sheet_name=sheet2_name)
            
            self._log_message(f"Лист 1: {len(df1)} строк, {len(df1.columns)} колонок")
            self._log_message(f"Лист 2: {len(df2)} строк, {len(df2.columns)} колонок")
            
            # Очищаем названия колонок
            df1 = self._clean_column_names(df1)
            df2 = self._clean_column_names(df2)
            key_column_clean = self._clean_column_names(pd.DataFrame(columns=[key_column])).columns[0]
            
            self._log_message(f"Ключевая колонка после очистки: '{key_column_clean}'")
            
            # Находим общие колонки (кроме ключевой)
            common_columns = list(set(df1.columns) & set(df2.columns) - {key_column_clean})
            self._log_message(f"Общие колонки для сравнения: {len(common_columns)}")
            
            if not common_columns:
                self._log_message("Нет общих колонок для сравнения!")
                return None, None, None
            
            if key_column_clean not in df1.columns or key_column_clean not in df2.columns:
                self._log_message(f"Ключевая колонка '{key_column_clean}' не найдена в обоих листах!")
                return None, None, None
            
            # Объединяем данные по ключевой колонке
            merged_df = pd.merge(df1, df2, on=key_column_clean, how='outer', 
                                suffixes=('_sheet1', '_sheet2'), indicator=True)
            
            self._log_message(f"После объединения: {len(merged_df)} строк")
            
            # Создаем DataFrame для результатов
            result_data = []
            
            # Подготавливаем структуру для результатов
            for idx, row in merged_df.iterrows():
                result_row = {}
                key_value = row[key_column_clean]
                result_row[key_column_clean] = key_value
                
                # Добавляем информацию о наличии в обоих листах
                merge_status = row['_merge']
                result_row['Статус'] = merge_status
                
                # Считаем количество расхождений
                diff_count = 0
                
                # Сравниваем каждую общую колонку
                for col in common_columns:
                    col1 = f"{col}_sheet1"
                    col2 = f"{col}_sheet2"
                    
                    val1 = row[col1] if col1 in row else None
                    val2 = row[col2] if col2 in row else None
                    
                    # Проверяем на расхождения
                    if pd.isna(val1) and pd.isna(val2):
                        # Оба значения NaN - считаем равными
                        are_different = False
                    elif pd.isna(val1) or pd.isna(val2):
                        # Один NaN, другой не NaN - разные
                        are_different = True
                    else:
                        # Оба не NaN - сравниваем значения
                        are_different = str(val1) != str(val2)
                    
                    # Сохраняем значения и флаг различия
                    result_row[f'{col}_sheet1'] = val1
                    result_row[f'{col}_sheet2'] = val2
                    result_row[f'{col}_diff'] = are_different
                    
                    if are_different:
                        diff_count += 1
                
                result_row['Количество_расхождений'] = diff_count
                result_data.append(result_row)
            
            # Создаем итоговый DataFrame
            result_df = pd.DataFrame(result_data)
            
            # Сортируем по количеству расхождений (по убыванию)
            result_df = result_df.sort_values('Количество_расхождений', ascending=False)
            
            self._log_message(f"Сравнение данных завершено. Всего строк: {len(result_df)}")
            self._log_message(f"Строк с расхождениями: {len(result_df[result_df['Количество_расхождений'] > 0])}")
            
            return result_df, common_columns, key_column_clean
            
        except Exception as e:
            self._log_message(f"Ошибка при сравнении данных: {e}")
            import traceback
            traceback.print_exc()
            return None, None, None

    def _apply_comparison_formatting(self, output_file, common_columns):
        """Применить форматирование к листу сравнения данных"""
        try:
            self._log_message("Применяем форматирование к сравнению данных...")
            
            # Загружаем книгу
            book = load_workbook(output_file)
            sheet = book["Сравнение_данных"]
            
            # Розовая заливка для различий
            pink_fill = PatternFill(start_color='FFB6C1', end_color='FFB6C1', fill_type='solid')
            
            # Находим индексы колонок в созданном файле
            col_mapping = {}
            for idx, cell in enumerate(sheet[1], 1):  # Первая строка с заголовками
                col_mapping[cell.value] = idx
            
            self._log_message(f"Найдено колонок в листе сравнения: {len(col_mapping)}")
            
            # Применяем форматирование для каждой строки
            for row_idx in range(2, sheet.max_row + 1):  # Начинаем со второй строки (после заголовка)
                for col in common_columns:
                    diff_col_name = f'{col}_diff'
                    if diff_col_name in col_mapping:
                        diff_cell = sheet.cell(row=row_idx, column=col_mapping[diff_col_name])
                        
                        # Проверяем булево значение
                        if diff_cell.value is True:
                            # Закрашиваем соответствующие ячейки с значениями
                            col1_name = f'{col}_sheet1'
                            col2_name = f'{col}_sheet2'
                            
                            if col1_name in col_mapping:
                                col1_cell = sheet.cell(row=row_idx, column=col_mapping[col1_name])
                                col1_cell.fill = pink_fill
                            
                            if col2_name in col_mapping:
                                col2_cell = sheet.cell(row=row_idx, column=col_mapping[col2_name])
                                col2_cell.fill = pink_fill
            
            # Скрываем колонки с флагами различий
            for col in common_columns:
                diff_col_name = f'{col}_diff'
                if diff_col_name in col_mapping:
                    col_idx = col_mapping[diff_col_name]
                    col_letter = self._get_column_letter(col_idx)
                    sheet.column_dimensions[col_letter].hidden = True
                    self._log_message(f"Скрыта колонка: {diff_col_name}")
            
            # Автоподгонка ширины колонок
            for col_idx in range(1, len(col_mapping) + 1):
                max_length = 0
                col_letter = self._get_column_letter(col_idx)
                for row_idx in range(1, sheet.max_row + 1):
                    cell = sheet.cell(row=row_idx, column=col_idx)
                    if cell.value is not None:
                        max_length = max(max_length, len(str(cell.value)))
                adjusted_width = min(max_length + 2, 50)
                sheet.column_dimensions[col_letter].width = adjusted_width
            
            book.save(output_file)
            self._log_message("Форматирование сравнения данных применено успешно")
            
        except Exception as e:
            self._log_message(f"Ошибка при форматировании сравнения данных: {e}")

    def _apply_column_report_formatting(self, output_file, sheet1_name, sheet2_name):
        """Применить форматирование к отчету по колонкам"""
        try:
            self._log_message("Применяем форматирование к отчету по колонкам...")
            
            # Загружаем книгу
            book = load_workbook(output_file)
            sheet = book["Отчет_по_колонкам"]
            
            # Цвета для разных статусов
            common_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')  # Зеленый
            only_sheet1_fill = PatternFill(start_color='FFEB9C', end_color='FFEB9C', fill_type='solid')  # Желтый
            only_sheet2_fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')  # Красный
            
            # Находим индексы колонок
            col_mapping = {}
            for idx, cell in enumerate(sheet[1], 1):  # Первая строка с заголовками
                col_mapping[cell.value] = idx
            
            # Применяем цветовое форматирование
            for row_idx in range(2, sheet.max_row + 1):
                status_cell = sheet.cell(row=row_idx, column=col_mapping['Статус'])
                status_value = status_cell.value
                
                fill_color = None
                if status_value == 'ОБЩАЯ':
                    fill_color = common_fill
                elif status_value == f'ТОЛЬКО В {sheet1_name}':
                    fill_color = only_sheet1_fill
                elif status_value == f'ТОЛЬКО В {sheet2_name}':
                    fill_color = only_sheet2_fill
                
                if fill_color:
                    # Закрашиваем всю строку
                    for col_idx in range(1, len(col_mapping) + 1):
                        sheet.cell(row=row_idx, column=col_idx).fill = fill_color
            
            # Автоподгонка ширины колонок
            for col_idx in range(1, len(col_mapping) + 1):
                max_length = 0
                col_letter = self._get_column_letter(col_idx)
                for row_idx in range(1, sheet.max_row + 1):
                    cell = sheet.cell(row=row_idx, column=col_idx)
                    if cell.value is not None:
                        max_length = max(max_length, len(str(cell.value)))
                adjusted_width = min(max_length + 2, 50)
                sheet.column_dimensions[col_letter].width = adjusted_width
            
            book.save(output_file)
            self._log_message("Форматирование отчета по колонкам применено успешно")
            
        except Exception as e:
            self._log_message(f"Ошибка при форматировании отчета по колонкам: {e}")

    def _get_column_letter(self, col_idx):
        """Конвертировать индекс колонки в букву"""
        letters = ''
        while col_idx > 0:
            col_idx, remainder = divmod(col_idx - 1, 26)
            letters = chr(65 + remainder) + letters
        return letters

    def full_comparison(self, sheet1_name, sheet2_name, key_column, output_file=None):
        """
        Полное сравнение: отчет по колонкам + сравнение данных + логи в одной книге
        """
        try:
            self._log_message("=== НАЧАЛО ПОЛНОГО СРАВНЕНИЯ ===")
            
            # Создаем имя выходного файла
            if output_file is None:
                output_file = self.file_path.replace('.xlsx', '_full_comparison.xlsx')
            
            # 1. Создаем отчет по колонкам
            self._log_message("\n1. АНАЛИЗ КОЛОНОК:")
            column_report = self.compare_column_names(sheet1_name, sheet2_name)
            
            # 2. Сравниваем данные
            self._log_message("\n2. СРАВНЕНИЕ ДАННЫХ:")
            data_report, common_columns, key_column_clean = self.compare_sheets(sheet1_name, sheet2_name, key_column)
            
            # 3. Сохраняем все в одну книгу БЕЗ форматирования
            self._log_message("\n3. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ:")
            
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                # Сохраняем отчет по колонкам
                if column_report is not None:
                    column_report.to_excel(writer, sheet_name="Отчет_по_колонкам", index=False)
                    self._log_message("Отчет по колонкам сохранен")
                
                # Сохраняем сравнение данных
                if data_report is not None:
                    data_report.to_excel(writer, sheet_name="Сравнение_данных", index=False)
                    self._log_message("Сравнение данных сохранено")
                
                # Сохраняем логи
                log_df = pd.DataFrame({'Логи': self.log_messages})
                log_df.to_excel(writer, sheet_name="Логи_выполнения", index=False)
                self._log_message("Логи выполнения сохранены")
            
            # 4. Применяем форматирование ПОСЛЕ сохранения
            self._log_message("\n4. ПРИМЕНЕНИЕ ФОРМАТИРОВАНИЯ:")
            
            # Форматируем отчет по колонкам
            if column_report is not None:
                self._apply_column_report_formatting(output_file, sheet1_name, sheet2_name)
            
            # Форматируем сравнение данных
            if data_report is not None and common_columns:
                self._apply_comparison_formatting(output_file, common_columns)
            
            # Восстанавливаем stdout
            self._restore_stdout()
            
            print(f"\n=== ПОЛНОЕ СРАВНЕНИЕ ЗАВЕРШЕНО ===")
            print(f"Результаты сохранены в: {output_file}")
            
            # Проверяем созданные листы
            book = load_workbook(output_file)
            print(f"Листы в книге: {book.sheetnames}")
            book.close()
            
            return {
                'output_file': output_file,
                'column_report': column_report,
                'data_report': data_report,
                'logs': self.log_messages
            }
            
        except Exception as e:
            self._log_message(f"Критическая ошибка при полном сравнении: {e}")
            self._restore_stdout()
            import traceback
            traceback.print_exc()
            return None

# Функция для быстрого использования
def quick_full_comparison(file_path, sheet1, sheet2, key_column):
    """
    Быстрое полное сравнение двух листов
    """
    print(f"Быстрое полное сравнение: {sheet1} vs {sheet2}")
    
    comparator = ExcelComparator(file_path)
    result = comparator.full_comparison(
        sheet1_name=sheet1,
        sheet2_name=sheet2,
        key_column=key_column
    )
    
    return result




In [ ]:
# Пример использования
if __name__ == "__main__":
    file_path = "шаблон\Стволы скважин сравнение.xlsx"
    
    try:
        result = quick_full_comparison(
            file_path=file_path,
            sheet1="КССС1",
            sheet2="КССС2",
            key_column="CSCD_ID"
        )
        
        if result:
            print(f"\nУСПЕХ! Файл создан: {result['output_file']}")
            print("Проверьте:")
            print("- Розовые ячейки в 'Сравнение_данных' для расхождений")
            print("- Цветные строки в 'Отчет_по_колонкам'")
            print("- Скрытые колонки *_diff в 'Сравнение_данных'")
        else:
            print("\nОШИБКА! Сравнение не выполнено")
            
    except Exception as e:
        print(f"Критическая ошибка: {e}")

## Поиск шаблонов и создание папок для них

In [9]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import re
import warnings
import io
import sys
from datetime import datetime
import os
from pathlib import Path
import shutil

In [26]:
def find_and_copy_excel_files(file_names, source_folder, target_folder):
    """
    Находит Excel файлы в исходной папке и копирует их в целевую папку
    """
    
    # Проверяем существование исходной папки
    if not os.path.exists(source_folder):
        print(f"❌ Ошибка: Исходная папка '{source_folder}' не существует")
        return
    
    print(f"✅ Исходная папка существует: {source_folder}")
    
    # Создаем целевую папку если она не существует
    os.makedirs(target_folder, exist_ok=True)
    print(f"✅ Целевая папка: {target_folder}")
    
    # Расширения Excel файлов
    excel_extensions = ['.xlsx', '.xls', '.xlsm', '.xlsb']
    
    found_files = 0
    all_excel_files = []
    
    print(f"\n🔍 Поиск файлов из списка: {file_names}")
    print("📂 Сканирую папки...")
    print("-" * 60)
    
    # Рекурсивно обходим все папки и подпапки
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            file_path = Path(root) / file
            file_name_without_ext = file_path.stem  # Имя без расширения
            file_ext = file_path.suffix.lower()     # Расширение в нижнем регистре
            
            # Собираем все Excel файлы для отладки
            if file_ext in excel_extensions:
                all_excel_files.append((file_name_without_ext, file_ext, str(file_path)))
            
            # Проверяем, является ли файл Excel файлом и есть ли он в списке
            if file_ext in excel_extensions and file_name_without_ext in file_names:
                print(f"🎯 НАЙДЕН ФАЙЛ: {file_name_without_ext}{file_ext}")
                print(f"   📍 Расположение: {file_path}")
                
                # Создаем путь для целевой папки
                target_subfolder = Path(target_folder) / file_name_without_ext
                
                try:
                    # Создаем целевую папку
                    os.makedirs(target_subfolder, exist_ok=True)
                    
                    # Копируем файл
                    target_file_path = target_subfolder / file
                    shutil.copy2(file_path, target_file_path)
                    
                    print(f"   ✅ СКОПИРОВАН: {target_file_path}")
                    found_files += 1
                    
                except Exception as e:
                    print(f"   ❌ Ошибка при копировании: {e}")
    
    # Выводим отладочную информацию
    print("\n" + "=" * 60)
    print("📊 ДИАГНОСТИЧЕСКАЯ ИНФОРМАЦИЯ:")
    print(f"📁 Всего найдено Excel файлов: {len(all_excel_files)}")
    
    if all_excel_files:
        print("\n📋 Список всех найденных Excel файлов:")
        for name, ext, path in all_excel_files[:20]:  # Показываем первые 20
            print(f"   - {name}{ext}")
        
        if len(all_excel_files) > 20:
            print(f"   ... и еще {len(all_excel_files) - 20} файлов")
        
        # Проверяем какие файлы из нашего списка не найдены
        print(f"\n🔎 Поиск файлов из вашего списка:")
        for search_name in file_names:
            found = any(search_name == name for name, ext, path in all_excel_files)
            status = "✅ НАЙДЕН" if found else "❌ НЕ НАЙДЕН"
            print(f"   {status}: {search_name}")
    
    print(f"\n📈 ИТОГ: Найдено и скопировано файлов: {found_files}")

def main():
    # Список названий Excel файлов (без расширения)
    excel_file_names = [
      'Пожарные извещатели и комплектующие к ним_1','Запчасти и узлы машин коммунал.хозяйст_1','Командоаппараты_4','Системы элобогрева и комплект. к ним_2','Бадьи и тара для бетона_1','Уст-ки днаплавки металла на пов-ть изд_1','Станции очистки хоз-быт сточных вод_1','Детали оборуд. дочистки трубопроводов_1','Установки осушки газа_1','Установки парогенерирующие_1','POS-оборудование_1','Части запасные для ТРК_1','Мебель и торговое оборудование для АЗС_1','Кронштейны_1'
    ]
    
    # Сетевая папка источник - ЗАМЕНИТЕ НА ВАШ ПУТЬ
    source_folder = r"\\mskpdcr3\SAP\KCCC\_Методика\шаблоны\Шаблоны_Ай-Теко\На загрузку"  # ЗАМЕНИТЕ!
    
    # Целевая папка - ЗАМЕНИТЕ НА ВАШ ПУТЬ
    target_folder = r"C:\Users\ZubarevVV\Desktop\Шаблоны"  # ЗАМЕНИТЕ!
    
    print("🚀 ЗАПУСК СКРИПТА ДЛЯ ПОИСКА EXCEL ФАЙЛОВ")
    print("=" * 60)
    
    # Выполняем поиск и копирование
    find_and_copy_excel_files(excel_file_names, source_folder, target_folder)

if __name__ == "__main__":
    main()



🚀 ЗАПУСК СКРИПТА ДЛЯ ПОИСКА EXCEL ФАЙЛОВ
✅ Исходная папка существует: \\mskpdcr3\SAP\KCCC\_Методика\шаблоны\Шаблоны_Ай-Теко\На загрузку
✅ Целевая папка: C:\Users\ZubarevVV\Desktop\Шаблоны

🔍 Поиск файлов из списка: ['Пожарные извещатели и комплектующие к ним_1', 'Запчасти и узлы машин коммунал.хозяйст_1', 'Командоаппараты_4', 'Системы элобогрева и комплект. к ним_2', 'Бадьи и тара для бетона_1', 'Уст-ки днаплавки металла на пов-ть изд_1', 'Станции очистки хоз-быт сточных вод_1', 'Детали оборуд. дочистки трубопроводов_1', 'Установки осушки газа_1', 'Установки парогенерирующие_1', 'POS-оборудование_1', 'Части запасные для ТРК_1', 'Мебель и торговое оборудование для АЗС_1', 'Кронштейны_1']
📂 Сканирую папки...
------------------------------------------------------------
🎯 НАЙДЕН ФАЙЛ: POS-оборудование_1.xlsx
   📍 Расположение: \\mskpdcr3\SAP\KCCC\_Методика\шаблоны\Шаблоны_Ай-Теко\На загрузку\Оборудование общепромышленное\POS-оборудование_1.xlsx
   ✅ СКОПИРОВАН: C:\Users\ZubarevVV\Desktop\Ш

## Разделение Excel на части

In [5]:
import pandas as pd
import os
from math import ceil

def split_excel_to_txt_preserve_format(input_file, output_dir="output_txt_files", chunk_size=1000, delimiter='\t'):
    """
    Разделяет Excel файл на несколько txt файлов по chunk_size записей каждый
    в формате UTF-16 LE с BOM, сохраняя точное текстовое представление данных
    """
    import codecs
    
    # Создаем директорию для выходных файлов, если она не существует
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        print(f"Загрузка файла: {input_file}")
        
        # Загружаем Excel файл, сохраняя оригинальные типы данных как строки
        df = pd.read_excel(input_file, dtype=str, keep_default_na=False)
        total_rows = len(df)
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        # Вычисляем количество файлов
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} txt файлов по {chunk_size} записей каждый")
        print(f"Формат: UTF-16 LE with BOM, разделитель: '{delimiter}'")
        print("Режим: сохранение точного текстового представления")
        
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            chunk_df = df.iloc[start_idx:end_idx]
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.txt")
            
            # Сохраняем с использованием codecs для гарантированного BOM
            with codecs.open(output_file, 'w', encoding='utf-16-le') as f:
                # Записываем BOM
                f.write('\ufeff')
                # Сохраняем DataFrame как CSV с сохранением формата
                chunk_df.to_csv(f, sep=delimiter, index=False, header=True, 
                              quoting=None, escapechar='\\', doublequote=False)
            
            print(f"Создан файл: {output_file} (записей: {len(chunk_df)})")
        
        print(f"\nГотово! Все txt файлы сохранены в формате UTF-16 LE with BOM")
        print("Все данные сохранены в точном текстовом представлении")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Альтернативная версия с более тонким контролем над типами данных
def split_excel_to_txt_preserve_format_v2(input_file, output_dir="output_txt_files", chunk_size=1000, delimiter='\t'):
    """
    Версия с ручной обработкой для максимального сохранения формата
    """
    import codecs
    from openpyxl import load_workbook
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        print(f"Загрузка файла: {input_file}")
        
        # Загружаем с помощью openpyxl для более точного контроля
        wb = load_workbook(input_file, data_only=False)
        ws = wb.active
        
        # Получаем все данные как есть
        all_data = []
        for row in ws.iter_rows(values_only=True):
            all_data.append(row)
        
        if not all_data:
            print("Файл пуст!")
            return
        
        headers = all_data[0]
        data_rows = all_data[1:]
        total_rows = len(data_rows)
        
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} txt файлов по {chunk_size} записей каждый")
        
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.txt")
            
            with codecs.open(output_file, 'w', encoding='utf-16-le') as f:
                f.write('\ufeff')  # BOM
                
                # Записываем заголовки
                f.write(delimiter.join(str(header) for header in headers) + '\n')
                
                # Записываем данные
                for j in range(start_idx, end_idx):
                    row = data_rows[j]
                    # Преобразуем все значения в строки, сохраняя точное представление
                    formatted_row = []
                    for cell in row:
                        if cell is None:
                            formatted_row.append('')
                        else:
                            # Сохраняем точное текстовое представление
                            formatted_row.append(str(cell))
                    
                    f.write(delimiter.join(formatted_row) + '\n')
            
            print(f"Создан файл: {output_file} (записей: {end_idx - start_idx})")
        
        print(f"\nГотово! Все данные сохранены в точном текстовом представлении")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Версия с использованием xlrd для лучшего сохранения форматов
def split_excel_to_txt_preserve_format_v3(input_file, output_dir="output_txt_files", chunk_size=1000, delimiter='\t'):
    """
    Версия с использованием pandas с дополнительными параметрами для сохранения формата
    """
    import codecs
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        print(f"Загрузка файла: {input_file}")
        
        # Загружаем с параметрами, сохраняющими текстовое представление
        df = pd.read_excel(
            input_file, 
            dtype=str,  # Все данные как строки
            keep_default_na=False,  # Не преобразовывать в NaN
            na_values=[],  # Не считать никакие значения как NA
            thousands=None,  # Отключить автоматическое форматирование чисел
            decimal='.'  # Явно указать разделитель десятичных
        )
        
        # Заменяем NaN на пустые строки
        df = df.fillna('')
        
        total_rows = len(df)
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} txt файлов по {chunk_size} записей каждый")
        
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            chunk_df = df.iloc[start_idx:end_idx]
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.txt")
            
            with codecs.open(output_file, 'w', encoding='utf-16-le') as f:
                f.write('\ufeff')  # BOM
                
                # Сохраняем с отключением всех преобразований
                chunk_df.to_csv(
                    f, 
                    sep=delimiter, 
                    index=False, 
                    header=True,
                    quoting=None,
                    escapechar='\\',
                    doublequote=False
                )
            
            print(f"Создан файл: {output_file} (записей: {len(chunk_df)})")
        
        print(f"\nГотово! Все данные сохранены в точном текстовом представлении")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Функция для проверки результатов
def verify_text_preservation(output_dir="output_txt_files", sample_records=3):
    """
    Проверяет сохранение текстового формата в созданных файлах
    """
    if not os.path.exists(output_dir):
        print("Директория не найдена!")
        return
    
    txt_files = [f for f in sorted(os.listdir(output_dir)) if f.endswith('.txt')]
    
    for file in txt_files[:2]:  # Проверяем первые 2 файла
        file_path = os.path.join(output_dir, file)
        print(f"\n{'='*60}")
        print(f"Проверка файла: {file}")
        print(f"{'='*60}")
        
        try:
            with open(file_path, 'r', encoding='utf-16-le') as f:
                lines = f.readlines()
                
            # Показываем заголовки и несколько записей
            for i, line in enumerate(lines[:sample_records + 1]):
                print(f"Строка {i}: {line.strip()}")
                
        except Exception as e:
            print(f"Ошибка при чтении файла: {e}")

# Интерактивная версия
def split_excel_preserve_format_interactive():
    """
    Интерактивная версия с сохранением формата
    """
    
    file_path = input("Введите путь к Excel файлу: ").strip()
    
    if not os.path.exists(file_path):
        print("Файл не найден!")
        return
    
    try:
        chunk_size = int(input("Введите количество записей в каждом файле (по умолчанию 1000): ") or "1000")
    except ValueError:
        chunk_size = 1000
        print("Используется значение по умолчанию: 1000")
    
    delimiter = input("Введите разделитель (по умолчанию табуляция '\\t'): ").strip()
    if delimiter == '\\t':
        delimiter = '\t'
    elif not delimiter:
        delimiter = '\t'
    
    output_dir = input("Введите директорию для сохранения (по умолчанию 'output_txt_files'): ").strip()
    if not output_dir:
        output_dir = "output_txt_files"
    
    # Выбираем версию
    print("\nВыберите метод обработки:")
    print("1. Базовая версия (рекомендуется)")
    print("2. Расширенная версия")
    print("3. Максимальное сохранение формата")
    
    choice = input("Введите номер (по умолчанию 1): ").strip()
    
    if choice == "2":
        split_excel_to_txt_preserve_format_v2(file_path, output_dir, chunk_size, delimiter)
    elif choice == "3":
        split_excel_to_txt_preserve_format_v3(file_path, output_dir, chunk_size, delimiter)
    else:
        split_excel_to_txt_preserve_format(file_path, output_dir, chunk_size, delimiter)
    
    # Проверяем результат
    verify_text_preservation(output_dir)

# Основное использование
if __name__ == "__main__":
    input_excel_file = "Блокировка_с_актуальными.xlsx"  # Укажите путь к вашему Excel файлу
    
    # Рекомендуемая версия
    split_excel_to_txt_preserve_format(input_excel_file, chunk_size=10000)
    
    # Проверяем результат
    verify_text_preservation()
    
    # Интерактивный режим
    # split_excel_preserve_format_interactive()


Загрузка файла: Блокировка_с_актуальными.xlsx
Файл загружен. Всего записей: 97400
Будет создано 10 txt файлов по 10000 записей каждый
Формат: UTF-16 LE with BOM, разделитель: '	'
Режим: сохранение точного текстового представления
Создан файл: output_txt_files\Блокировка_с_актуальными_part_1.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_2.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_3.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_4.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_5.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_6.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_7.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_8.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_с_актуальными_part_9.txt (записей: 10000)
Создан файл: output_txt

In [2]:
# НЕ ИСПОЛЬЗОВАТЬ - НЕВЕРНЫЙ ФОРМАТ БЛОКИРОВКИ
import pandas as pd
import os
from math import ceil

def split_excel_to_txt(input_file, output_dir="output_txt_files", chunk_size=1000, delimiter='\t'):
    """
    Разделяет Excel файл на несколько txt файлов по chunk_size записей каждый
    в формате UTF-16 LE с BOM
    
    Parameters:
    input_file (str): путь к исходному Excel файлу
    output_dir (str): директория для сохранения разделенных файлов
    chunk_size (int): количество записей в каждом файле (по умолчанию 1000)
    delimiter (str): разделитель для txt файлов (по умолчанию табуляция)
    """
    
    # Создаем директорию для выходных файлов, если она не существует
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        # Загружаем Excel файл
        print(f"Загрузка файла: {input_file}")
        df = pd.read_excel(input_file)
        total_rows = len(df)
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        # Вычисляем количество файлов
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} txt файлов по {chunk_size} записей каждый")
        print(f"Формат: UTF-16 LE with BOM, разделитель: '{delimiter}'")
        
        # Разделяем DataFrame на части и сохраняем
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            # Выбираем часть данных
            chunk_df = df.iloc[start_idx:end_idx]
            
            # Формируем имя файла
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.txt")
            
            # Сохраняем часть в txt с UTF-16 LE BOM
            chunk_df.to_csv(
                output_file, 
                sep=delimiter, 
                index=False,
                encoding='utf-16-le',  # UTF-16 Little Endian
                lineterminator='\n'
            )
            
            # Добавляем BOM вручную, так как pandas может не добавлять его автоматически
            with open(output_file, 'r+b') as f:
                content = f.read()
                f.seek(0)
                f.write(b'\xff\xfe')  # UTF-16 LE BOM
                f.write(content)
            
            print(f"Создан файл: {output_file} (записей: {len(chunk_df)})")
        
        print(f"\nГотово! Все txt файлы сохранены в директории: {output_dir}")
        print(f"Формат: UTF-16 LE with BOM")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Альтернативная версия с использованием codecs для более надежного BOM
def split_excel_to_txt_v2(input_file, output_dir="output_txt_files", chunk_size=1000, delimiter='\t'):
    """
    Альтернативная версия с использованием codecs для надежного добавления BOM
    """
    import codecs
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        print(f"Загрузка файла: {input_file}")
        df = pd.read_excel(input_file)
        total_rows = len(df)
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} txt файлов по {chunk_size} записей каждый")
        
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            chunk_df = df.iloc[start_idx:end_idx]
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.txt")
            
            # Сохраняем с использованием codecs для гарантированного BOM
            with codecs.open(output_file, 'w', encoding='utf-16-le') as f:
                # Записываем BOM
                f.write('\ufeff')
                # Сохраняем DataFrame
                chunk_df.to_csv(f, sep=delimiter, index=False, lineterminator='\n')
            
            print(f"Создан файл: {output_file} (записей: {len(chunk_df)})")
        
        print(f"\nГотово! Все txt файлы сохранены в формате UTF-16 LE with BOM")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Интерактивная версия
def split_excel_to_txt_interactive():
    """
    Интерактивная версия для Jupyter Notebook
    """
    
    # Запрос пути к файлу
    file_path = input("Введите путь к Excel файлу: ").strip()
    
    if not os.path.exists(file_path):
        print("Файл не найден!")
        return
    
    # Запрос размера чанка
    try:
        chunk_size = int(input("Введите количество записей в каждом файле (по умолчанию 1000): ") or "1000")
    except ValueError:
        chunk_size = 1000
        print("Используется значение по умолчанию: 1000")
    
    # Запрос разделителя
    delimiter = input("Введите разделитель (по умолчанию табуляция '\\t'): ").strip()
    if delimiter == '\\t':
        delimiter = '\t'
    elif not delimiter:
        delimiter = '\t'
    
    # Запрос директории для сохранения
    output_dir = input("Введите директорию для сохранения (по умолчанию 'output_txt_files'): ").strip()
    if not output_dir:
        output_dir = "output_txt_files"
    
    # Выполняем разделение
    split_excel_to_txt_v2(file_path, output_dir, chunk_size, delimiter)

# Функция для проверки кодировки файла
def check_file_encoding(file_path):
    """
    Проверяет кодировку созданного файла
    """
    try:
        with open(file_path, 'rb') as f:
            bom = f.read(2)
            if bom == b'\xff\xfe':
                print(f"✓ {file_path} - UTF-16 LE BOM подтвержден")
            else:
                print(f"✗ {file_path} - BOM не найден, первые байты: {bom.hex()}")
    except Exception as e:
        print(f"Ошибка при проверке файла {file_path}: {e}")

# Функция для проверки всех созданных файлов
def verify_all_files(output_dir="output_txt_files"):
    """
    Проверяет кодировку всех созданных файлов
    """
    if not os.path.exists(output_dir):
        print("Директория не найдена!")
        return
    
    print("Проверка кодировки файлов:")
    for file in sorted(os.listdir(output_dir)):
        if file.endswith('.txt'):
            file_path = os.path.join(output_dir, file)
            check_file_encoding(file_path)

# Использование функций
if __name__ == "__main__":
    # Основное использование
    input_excel_file = "Блокировка_с_актуальными.xlsx"  # Укажите путь к вашему Excel файлу
    
    # Версия 1 (простая)
    # split_excel_to_txt(input_excel_file, chunk_size=1000)
    
    # Версия 2 (с codecs - рекомендуется)
    split_excel_to_txt_v2(input_excel_file, chunk_size=10000)
    
    # Проверка кодировки
    verify_all_files()
    
    # Интерактивный режим
    # split_excel_to_txt_interactive()


Загрузка файла: Блокировка.xlsx
Файл загружен. Всего записей: 167934
Будет создано 17 txt файлов по 10000 записей каждый
Создан файл: output_txt_files\Блокировка_part_1.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_2.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_3.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_4.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_5.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_6.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_7.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_8.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_9.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_10.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_11.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_12.txt (записей: 10000)
Создан файл: output_txt_files\Блокировка_part_13

In [2]:
import pandas as pd
import os
from math import ceil

def split_excel_file(input_file, output_dir="output_excel_files_anton", chunk_size=10000):
    """
    Разделяет Excel файл на несколько файлов по chunk_size записей каждый
    
    Parameters:
    input_file (str): путь к исходному Excel файлу
    output_dir (str): директория для сохранения разделенных файлов
    chunk_size (int): количество записей в каждом файле (по умолчанию 1000)
    """
    
    # Создаем директорию для выходных файлов, если она не существует
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")
    
    try:
        # Загружаем Excel файл
        print(f"Загрузка файла: {input_file}")
        df = pd.read_excel(input_file)
        total_rows = len(df)
        print(f"Файл загружен. Всего записей: {total_rows}")
        
        # Вычисляем количество файлов
        num_files = ceil(total_rows / chunk_size)
        print(f"Будет создано {num_files} файлов по {chunk_size} записей каждый")
        
        # Разделяем DataFrame на части и сохраняем
        for i in range(num_files):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            # Выбираем часть данных
            chunk_df = df.iloc[start_idx:end_idx]
            
            # Формируем имя файла
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.xlsx")
            
            # Сохраняем часть в Excel
            chunk_df.to_excel(output_file, index=False)
            print(f"Создан файл: {output_file} (записей: {len(chunk_df)})")
        
        print(f"\nГотово! Все файлы сохранены в директории: {output_dir}")
        
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Использование функции
# Замените 'your_file.xlsx' на путь к вашему файлу
input_excel_file = "KONTRAGENTY.xlsx"  # Укажите путь к вашему Excel файлу

# Вызов функции
split_excel_file(input_excel_file, chunk_size=10000)


Создана директория: output_excel_files_anton
Загрузка файла: KONTRAGENTY.xlsx
Файл загружен. Всего записей: 675975
Будет создано 68 файлов по 10000 записей каждый
Создан файл: output_excel_files_anton\KONTRAGENTY_part_1.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_2.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_3.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_4.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_5.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_6.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_7.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_8.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_9.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGENTY_part_10.xlsx (записей: 10000)
Создан файл: output_excel_files_anton\KONTRAGE